In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import requests as requests
from xml.etree import ElementTree

In [ ]:
# Read in excel file
depth_measurements_df = pd.read_excel('tidal_sample.xlsx',
                                      usecols=['Date', 'Time', 'GPS Latitude', 'GPS Longitude', 'depth'],
                                      na_values='<Null>')

In [ ]:
# Data preparation

In [ ]:
# Drop all rows where all column values are NA
depth_measurements_df = depth_measurements_df.dropna(axis='index', how='all').reset_index(drop=True)

In [ ]:
# Check data types
depth_measurements_df.info()

In [ ]:
# Replace commas with points and convert to floats
for col in ['GPS Latitude', 'GPS Longitude', 'depth']:
    depth_measurements_df[col] = depth_measurements_df[col].astype('str')
    depth_measurements_df[col] = depth_measurements_df[col].str.replace(',', '.')
    depth_measurements_df[col] = pd.to_numeric(depth_measurements_df[col], errors='coerce')

In [ ]:
# Combine columns date and time and set as datetime index
depth_measurements_df['Date_Time'] = pd.to_datetime(depth_measurements_df['Date'] + ' ' + depth_measurements_df['Time'])
depth_measurements_df.set_index('Date_Time', inplace=True)

In [ ]:
# Check for possible GPS outliers
depth_measurements_df['GPS Latitude'].hist()

In [ ]:
depth_measurements_df['GPS Longitude'].hist()

In [ ]:
# Check how many observations that might have reversed lat/lon
depth_measurements_df[(depth_measurements_df['GPS Latitude'] < 50) & (depth_measurements_df['GPS Longitude'] > 20)]

In [ ]:
# It seem as if lat/lon has been reversed in one observation. Reverse them.
reversed_lat_mask = depth_measurements_df['GPS Latitude'] < 50
lon = depth_measurements_df[reversed_lat_mask]['GPS Latitude']

reversed_lon_mask = depth_measurements_df['GPS Longitude'] > 20
lat = depth_measurements_df[reversed_lon_mask]['GPS Longitude']

depth_measurements_df.loc[reversed_lat_mask, 'GPS Latitude'] = lat
depth_measurements_df.loc[reversed_lon_mask, 'GPS Longitude'] = lon

In [ ]:
# Check distributions again
depth_measurements_df['GPS Latitude'].hist()

In [ ]:
depth_measurements_df['GPS Longitude'].hist()

In [ ]:
'''
Use the API from the Norwegian Mapping Authority to fetch the water levels with reference to chart datum.
The dynamic parameters being passed in for each observation inlude lat, long, fromtime (time of observation),
totime (time of observation + 1 hour). From the list of water levels returned from the API, we're adding the first (closest in time)
water level value to a new column (water_level). In addtion we're adding the time of the water level in a new column (water_level_time).
'''

In [ ]:
def parse_xml(content):
    """Get the first water level value and its time.
        
        Keyword arguments:
        content - xml content
    """
    water_level = None
    water_level_time = None
    
    try:
        root = ElementTree.fromstring(content)
        first_water_level = root.find('locationdata').find('data').find('waterlevel')
        water_level = first_water_level.attrib['value']
        water_level_time = first_water_level.attrib['time']
    except:
        pass
    return water_level, water_level_time

In [ ]:
def get_water_level(row):
    """Get the xml water levels using external api from the
        Norwegian Mapping Authority and assign to new columns
        
        Keyword arguments:
        content - pandas row
    """
    
    payload = {'tide_request': 'locationdata',
               'lat': row['GPS Latitude'],
               'lon': row['GPS Longitude'],
               'fromtime': row.name,
               'totime': row.name + pd.DateOffset(hours=1),
               'datatype': 'obs',
               'refcode': 'cd',
               'interval': '10',
               'lang': 'nn'}
    water_response = requests.get('http://api.sehavniva.no/tideapi.php', params=payload)
    
    water_level = None;
    water_level_time = None
    if water_response.status_code == 200:
        water_level, water_level_time = parse_xml(water_response.content)

    row['water_level'] = water_level
    row['water_level_time'] = water_level_time
    return row

In [ ]:
depth_measurements_df = depth_measurements_df.apply(get_water_level, axis=1)

In [ ]:
depth_measurements_df.head()

In [ ]:
depth_measurements_df['water_level'] = pd.to_numeric(depth_measurements_df['water_level'], errors='coerce')

In [ ]:
# Calculate chart datum referenced sampling depth for each observation
depth_measurements_df['chart_datum_adj_depth'] = depth_measurements_df['water_level'] - depth_measurements_df['depth']

In [ ]:
depth_measurements_df.head()

In [ ]:
# Export data to csv and excel files
out_csv = 'cd_adjusted_depth_measurements.csv'
depth_measurements_df.to_csv(out_csv, index=False)

out_xlsx = 'cd_adjusted_depth_measurements.xlsx'
depth_measurements_df.to_excel(out_xlsx, index=False)